<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/TP1/C%2B%2B/TP1_C%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TP1 en C++**

Utilizo el comando writefile para generar un archivo llamado tp1.cpp. Necesitaré ese archivo para crear un ejecutable. 

In [9]:
%%writefile tp1.cpp
#include <iostream>
#include <string>
#include <sys/wait.h>
#include <unistd.h>

#define HIJOS_DE_A 3
#define HIJOS_DE_B 2
#define HIJOS_DE_C 0
#define HIJOS_DE_D 1
#define HIJOS_DE_E HIJOS_DE_C
#define HIJOS_DE_F HIJOS_DE_E
#define HIJOS_DE_G HIJOS_DE_F

#define PROCESO_B 0
#define PROCESO_C 1
#define PROCESO_E 0

#define TIEMPO_DORMIDO 30
#define INICIO_ITERADOR 0
#define PROCESO_CREADO_CORRECTAMENTE 0
#define PROCESO_SIN_HIJOS 0
#define SALIDA_EXITOSA EXIT_SUCCESS
#define SALIDA_ERRONEA EXIT_FAILURE

bool procesoCreadoCorrectamente(pid_t pid);
void esperarPorMisHijos(int numeroHijos);
void mostrarMiInformacion(char letra, pid_t miPid, pid_t miParentPid);

bool procesoCreadoCorrectamente(pid_t pid)
{
  return pid >= PROCESO_CREADO_CORRECTAMENTE;
}

void esperarPorMisHijos(int numeroHijos)
{
  for (int i = INICIO_ITERADOR; i < numeroHijos; i++)
  {
    wait(NULL);
  }
}

void mostrarMiInformacion(char letra, pid_t miPid, pid_t miParentPid)
{
  std::cout<<"Soy el proceso "<<letra<<". Mi PID es "<<miPid<<" y el PID de mi padre es "<<miParentPid<<std::endl;
}

//Clase IProceso (simula una Interfaz)
class IProceso 
{
  protected:
    pid_t _miPid;
    pid_t _miPPid;
    char _letra;
    int _cantidadHijos;

  public:
    IProceso(pid_t pid, pid_t ppid, char caracter, int hijos) : _miPid(pid), _miPPid(ppid), _letra(caracter), _cantidadHijos(hijos) 
    {
    }
    virtual void realizarAccion() = SALIDA_EXITOSA;
};

//Procesos que tienen hijos y deben esperarlos
class ProcesoConHijos : public IProceso 
{
  public:
    ProcesoConHijos(pid_t pid, pid_t ppid, char caracter, int hijos) : IProceso(pid, ppid, caracter, hijos) 
    {
    }
    void realizarAccion() override 
    {
      mostrarMiInformacion(this->_letra,this->_miPid,this->_miPPid);
      esperarPorMisHijos(this->_cantidadHijos);
    }
};

//Procesos que no tienen hijos y deben hacer sleep
class ProcesoSinHijos : public IProceso 
{
  public:
    ProcesoSinHijos(pid_t pid, pid_t ppid, char caracter, int hijos) : IProceso(pid, ppid, caracter, hijos) 
    {
    }
    void realizarAccion() override 
    {
      mostrarMiInformacion(this->_letra,this->_miPid,this->_miPPid);
      sleep(TIEMPO_DORMIDO);
    }
};

class ProcesoFabrica
{
  public:
    IProceso* determinarTipoProceso(pid_t pid, pid_t ppid, char caracter, int hijos)
    {
      if (hijos > PROCESO_SIN_HIJOS)
      {
        proceso = new ProcesoConHijos(pid,ppid,caracter,hijos);
      }
      else
      {
        proceso = new ProcesoSinHijos(pid,ppid,caracter,hijos);
      }
      return proceso;
    }
  private:
    IProceso* proceso;
};

int main(int argc, char *argv[])
{
  pid_t pidProcesoA, pidProcesoB, pidProcesoD;
  ProcesoFabrica fabricaDeProcesos;

  for (int i = INICIO_ITERADOR; i < HIJOS_DE_A; i++)
  {
    pidProcesoA = fork();
    if (!procesoCreadoCorrectamente(pidProcesoA))
    {
      std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
      return SALIDA_ERRONEA;
    }
    if (!pidProcesoA)
    {
      if (i == PROCESO_B)
      {                
        for (int j = INICIO_ITERADOR; j < HIJOS_DE_B; j++)
        {
          pidProcesoB = fork();
          if (!procesoCreadoCorrectamente(pidProcesoB))
          {
            std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
            return SALIDA_ERRONEA;
          }
          if (!pidProcesoB)
          {
            if (j == PROCESO_E)
            {
              IProceso* procesoE = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'E',HIJOS_DE_E);
              procesoE->realizarAccion();
              return SALIDA_EXITOSA;
            }
            else
            {
              IProceso* procesoF = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'F',HIJOS_DE_F);
              procesoF->realizarAccion();
              return SALIDA_EXITOSA;
            }
          }                
        }
        IProceso* procesoB = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'B',HIJOS_DE_B);
        procesoB->realizarAccion();
        return SALIDA_EXITOSA;
      }
      else
      {
        if (i == PROCESO_C)
        {
          IProceso* procesoC = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'C',HIJOS_DE_C);
          procesoC->realizarAccion();
          return SALIDA_EXITOSA;
        }
        else
        {
          pidProcesoD = fork();
          if (!procesoCreadoCorrectamente(pidProcesoD))
          {
            std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
            return SALIDA_ERRONEA;
          }
          if (!pidProcesoD)
          {
            IProceso* procesoG = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'G',HIJOS_DE_G);
            procesoG->realizarAccion();
            return SALIDA_EXITOSA;
          }
          IProceso* procesoD = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'D',HIJOS_DE_D);
          procesoD->realizarAccion();
          return SALIDA_EXITOSA;
        }
      }    
    }
  }
  IProceso* procesoA = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'A',HIJOS_DE_A);
  procesoA->realizarAccion();
  return SALIDA_EXITOSA;
}

Overwriting tp1.cpp


Listo los archivos para ver que el archivo tp1.cpp esté creado correctamente.

In [10]:
!ls -la

total 48
drwxr-xr-x 1 root root  4096 Apr 19 02:22 .
drwxr-xr-x 1 root root  4096 Apr 19 02:20 ..
drwxr-xr-x 4 root root  4096 Apr 14 13:34 .config
-rw-r--r-- 1 root root   418 Apr 19 02:22 salidaC
drwxr-xr-x 1 root root  4096 Apr 14 13:35 sample_data
-rwxr-xr-x 1 root root 19112 Apr 19 02:22 tp1
-rw-r--r-- 1 root root  5203 Apr 19 02:42 tp1.cpp


Compilo el archivo tp1.cpp para obtener un archivo a ejecutar. Este archivo se llamará tp1 y no llevará extensión. 

In [11]:
!g++ -std=c++11 -o tp1 tp1.cpp

Listo los archivos nuevamente para verificar que tengo el ejecutable del proceso. 

In [12]:
!ls -la

total 48
drwxr-xr-x 1 root root  4096 Apr 19 02:42 .
drwxr-xr-x 1 root root  4096 Apr 19 02:20 ..
drwxr-xr-x 4 root root  4096 Apr 14 13:34 .config
-rw-r--r-- 1 root root   418 Apr 19 02:22 salidaC
drwxr-xr-x 1 root root  4096 Apr 14 13:35 sample_data
-rwxr-xr-x 1 root root 19112 Apr 19 02:42 tp1
-rw-r--r-- 1 root root  5203 Apr 19 02:42 tp1.cpp


Utilizo el comando nohup para ejecutar el proceso en segundo plano, de manera que no me ocupe la consola. Es necesario añadir el caracter '&' al final.

Para ejecutar el proceso, tengo que poner ./tp1.

El comando 1>salidaC va a redirigir la salida estandar de la pantalla (stdout) a un archivo llamado salidaC.

El comando 2>/dev/null envía la salida de error estandar (stderr) a dev/null

In [13]:
!nohup ./tp1 1>salidaC 2>/dev/null & 

Al ejecutar el proceso, podré ver al proceso padre y a sus procesos hijos con el comando ps. 

In [14]:
!ps -ef | grep tp1

root        5812       1  0 02:42 ?        00:00:00 ./tp1
root        5813    5812  0 02:42 ?        00:00:00 ./tp1
root        5814    5812  0 02:42 ?        00:00:00 ./tp1
root        5815    5812  0 02:42 ?        00:00:00 ./tp1
root        5816    5813  0 02:42 ?        00:00:00 ./tp1
root        5817    5813  0 02:42 ?        00:00:00 ./tp1
root        5818    5815  0 02:42 ?        00:00:00 ./tp1
root        5831     518  0 02:42 ?        00:00:00 /bin/bash -c ps -ef | grep tp1
root        5833    5831  0 02:42 ?        00:00:00 grep tp1


Utilizo el comando pstree para poder obtener el árbol de procesos. Si le paso el PID del proceso padre, podré ver tanto su PID como el PID de sus hijos y nietos. 

In [15]:
!pstree -p 5812

tp1(5812)─┬─tp1(5813)─┬─tp1(5816)
          │           └─tp1(5817)
          ├─tp1(5814)
          └─tp1(5815)───tp1(5818)


Utilizo el comando cat para poder mostrar el contenido del archivo salidaC, que es a donde se fueron redirigidos los print que realizan cada uno de los procesos. 

In [16]:
!cat salidaC

Soy el proceso A. Mi PID es 5812 y el PID de mi padre es 1
Soy el proceso B. Mi PID es 5813 y el PID de mi padre es 5812
Soy el proceso F. Mi PID es 5817 y el PID de mi padre es 5813
Soy el proceso C. Mi PID es 5814 y el PID de mi padre es 5812
Soy el proceso E. Mi PID es 5816 y el PID de mi padre es 5813
Soy el proceso D. Mi PID es 5815 y el PID de mi padre es 5812
Soy el proceso G. Mi PID es 5818 y el PID de mi padre es 5815
